In [1]:
!echo "Hello, Bash!"


"Hello, Bash!"


In [2]:
!pip install openai streamlit pandas


In [3]:
!pip install spacy



In [4]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07
     --------------------------------------- 0.0/12.8 MB 100.9 kB/s eta 0:02:07


In [5]:
pip install -r requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [6]:
%%writefile tools.py

import requests
import random

def search_products(query, max_price, size):
    api_url = "https://fakestoreapi.com/products"
    response = requests.get(api_url)
    if response.status_code == 200:
        products = response.json()
        return [p for p in products if p['price'] <= max_price and size in p.get('sizes', [])]
    return []

def estimate_shipping(location, delivery_date):
    return {"cost": round(random.uniform(4.99, 15.99), 2), "delivery_date": delivery_date, "feasible": True}

def apply_discount(price, promo_code):
    discounts = {"SAVE10": 0.10, "WELCOME5": 0.05, "FASHION20": 0.20}
    discount_amount = price * discounts.get(promo_code, 0)
    return price - discount_amount

def compare_prices(product_name):
    competitors = ["Amazon", "eBay", "Walmart", "BestBuy"]
    return {store: round(random.uniform(40, 100), 2) for store in competitors}

def check_return_policy(site):
    policies = {"Amazon": "30-day return policy", "eBay": "No returns", "Walmart": "15-day return policy", "BestBuy": "45-day return policy"}
    return policies.get(site, "No return policy found.")

Overwriting tools.py


In [7]:
%%writefile agent.py
import tools
import spacy

def agent_response(user_query):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(user_query.lower())
    
    if "search" in user_query or "find" in user_query:
        return tools.search_products("Floral Skirt", 40, "S")
    elif "shipping" in user_query or "delivery" in user_query:
        return tools.estimate_shipping("New York", "2024-02-09")
    elif "discount" in user_query or "promo" in user_query:
        return tools.apply_discount(40, "SAVE10")
    elif "compare" in user_query or "cheaper" in user_query:
        return tools.compare_prices("Casual Denim Jacket")
    elif "return policy" in user_query or "returns" in user_query:
        return tools.check_return_policy("Amazon")
    return "I don't understand. Try asking about product search, shipping, discounts, price comparison, or return policies."



Overwriting agent.py


In [8]:
%%writefile streamlit_app.py

import subprocess
import sys

# Install spaCy model if not already installed
try:
    import spacy
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "spacy"])

# Download the model if not already downloaded
try:
    import en_core_web_sm
except ImportError:
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])



import streamlit as st
import agent  

st.set_page_config(page_title="AI Fashion Assistant", layout="wide")

st.title("🛍️ AI Fashion Shopping Assistant")
st.write("Ask me about product searches, shipping estimates, discounts, price comparisons, or return policies!")

query = st.text_input("Enter your request:")
if st.button("Submit"):
    response = agent.agent_response(query)
    
    if isinstance(response, list) and response:
        st.subheader("Here are some matching products:")
        for item in response:
            st.image("https://via.placeholder.com/150", width=100)
            st.write(f"**{item['title']}** - ${item['price']}")
    elif isinstance(response, dict):
        st.json(response)
    else:
        st.write(response)



Overwriting streamlit_app.py


In [ ]:
!streamlit run streamlit_app.py
